In [ ]:
# LOAD LIBRARIES

import cobra
import numpy as np
import pandas as pd
import cobra.test
from cobra.test import create_test_model
from cobra.sampling import sample
import random
import sys

In [ ]:
# SIMULATE FLUXOME AND TRANSCRIPTOME

# Random selection of reactions whose gene expressions will be shuffled
def randomizeTranscriptome(Transcriptome,flux,condition,reactionIndexes,ns):
    selected = random.sample( reactionIndexes , ns) # selected to have expression proportional to flux
    set_selected = set(selected)
    noSelected = [x for x in reactionIndexes if x not in set_selected] # not selected to have expression proportional to flux
    shuffled = noSelected.copy()
    random.shuffle(shuffled) # shuffling list
    for i in selected:
        Transcriptome[condition].iloc[i]=abs(flux[condition].iloc[i]) 
    for index,i in enumerate(noSelected):
        j = shuffled[index]
        Transcriptome[condition].iloc[i]=abs(flux[condition].iloc[j])
    return(Transcriptome)

# Simulate data

for Lambda in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
    # Set random seed
    randomSeed =   int(Lambda)
    random.seed(randomSeed)
    
    
    # Load iJO1366 network
    model_default = cobra.test.create_test_model("ecoli")
    # Fix biomass growht rate
    model_default.reactions.BIOMASS_Ec_iJO1366_core_53p95M.bounds=(0.98,0.98) 
    # No prior information in the form of bounds
    model_default.reactions.ATPM.bounds=(0,1000)
    model_default.reactions.EX_cbl1_e.bounds=(-1000,1000)

    # 1. Simulate 1000 fluxomes
    # Sample from fluxome space
    flux = sample(model_default, 1000)

    # 2. Simulate 1000 transcriptomes
    # Set gene names a reaction names
    geneNames = flux.columns.tolist()
    flux=flux.T

    # Add column names
    conditions = ["Simulation_"+str(i+1) for i in range(flux.shape[1])]
    flux.columns = conditions
    Transcriptome=flux.copy()

    # Shuffle a fraction of the transcriptome
    ns=round(len(model_default.reactions)*Lambda) # Number to shuffle
    reactionIndexes = range(len(model_default.reactions))
    for condition in conditions:
         randomizeTranscriptome(Transcriptome,flux,condition,reactionIndexes,ns)
    flux["Reaction"]    = geneNames
    Transcriptome["Gene"] = geneNames

    print("Lambda: ", Lambda,"===================")
    # Lambdas equal 0.0 and 1.0, are added to the file names as 0 and 1, respectively
    if Lambda == 0.0 or Lambda == 1.0: Lambda=int(Lambda) 
    # Save simulated Transcriptome and flux vectors as text files
    Transcriptome.to_csv('simulatedData/Ecoli_iJO1366_simulatedTranscriptome_Lambda_'+str(Lambda)+'.txt',sep="\t",index=False)
    flux.to_csv('simulatedData/Ecoli_iJO1366_simulatedFluxome_Lambda_'+str(Lambda)+'.txt',sep="\t",index=False)